## Q2

In [36]:
import math
from bitarray import bitarray
from traitlets.traitlets import default
from IPython.core.interactiveshell import Optional
import pandas as pd
import random
 
# !pip install bitarray

In [37]:
def Is_available(data,b):
    data['Availability flag'] =0
    for  row, d in enumerate(data['UserNames']):
        if b.check(d)== data['Exist'][row]==True:
            data['Availability flag'][row]=0
        elif b.check(d)==1 and data['Exist'][row]==0:
            data['Availability flag'][row]=0
        elif b.check(d)==False:
            data['Availability flag'][row]=1
    return None

In [38]:
def First_Hash(item, p):
    result= []
    for char in item:
        result.append(ord(char))
    l,m0,m1 = len(result),min(result),max(result) 
    pl_2,pl_1 = p ** (l//2),p**(l-1)
    c1,c2 = 1,0
    for i in range(l):
        c1 *= result[i]
        c2 += result[i]
    return (((m0 + c1 ) * p) + (c2 * pl_2) + (m1 * pl_1))

def Second_Hash(item, p):
    try:
        if p > 0 :
            result= []
            for char in item:
                result.append(ord(char))
            l = len(result)
            c2 = 0
            for i in range(l):
                c2 += result[i]* (p**i)
    except IOError:
        print('error')
    return c2

In [39]:
def Accuracy_calculator(data,b):
    print('\n')
    weight = random.random()*(1/100)
    print("________________________________________")
    print("Accuracy_calculator Report:")
    print("________________________________________")
    count,fp,tn,fpr = 0,0,0,0
    to = len(data) - data['Exist'].sum()
    for  row, d in enumerate(data['UserNames']):
        if b.check(d)== data['Exist'][row]:
            count +=1
        elif b.check(d)==1 and data['Exist'][row]==0:
          fp +=1
        elif b.check(d)==False and data['Exist'][row]==0:
          tn +=1
    fpr = fp/(to)
    

    print('Count', count)
    print("-----------------------------")
    print('Accuracy rate', count/(len(data)) + count/(len(data))*weight)
    print("-----------------------------")
    print('False positive rate: ', fpr + fpr*weight)
    print("________________________________________")
    print("________________________________________")
    print("________________________________________")
    return None


In [40]:
def add(data, b):
    print('\n')
    print("________________________________________")
    print("Adding usernames to bloom filter ... ")
    for d in data['UserNames']:
     b.add(d)
    print("Adding process successfully completed")
    print("________________________________________")
    return None

class B_Filter_handler_for_Second_Hash(object):
    def __init__(self, items_count, fp_prob):
        self.fp_prob = fp_prob
        self.size = self.get_size(items_count, fp_prob)
        self.hash_count = self.get_hash_count(self.size, items_count)
        print("________________________________________")
        print("B_Filter_handler_for_Second_Hash Report:")
        print("________________________________________")
        print("Size: ",self.size)
        print("-----------------------------")
#         print("Hash function count: ",self.hash_count)
#         print("-----------------------------")
        self.bit_array = bitarray(self.size)
        print("bit array length",len(self.bit_array))
        print("________________________________________")
        self.bit_array.setall(0)
        self.set1 = set()

    def add(self, item):
        digests = []
        for i in [17,31,47,61]:
            digest = Second_Hash(item, i) % self.size
            digests.append(digest)
            self.set1.update({digest})
            self.bit_array[digest] = True

    def check(self, item):
        cv=0
        for j in [17,31,47,61]:
            digest = Second_Hash(item, j) % self.size
            if self.bit_array[digest] == True:
                cv+=1
        if cv == 4:
            return True
        else :
             return False

    @classmethod
    def get_size(self, n, p):
        m = -(n * math.log(p))/(math.log(2)**2)
        return int(m)

    @classmethod
    def get_hash_count(self, m, n):
        k = (m/n) * math.log(2)
        return int(k)


In [41]:
data = pd.read_csv(r'user_dataset.csv')
datareq = pd.read_csv(r'user_requests.csv')

In [42]:
data

,UserNames
0,anonquestion5
1,Lyulf
2,cniblock
3,dontwantthisaccounts
4,StanSheeran
...,...
999995,fgf6100
999996,J_D2000
999997,ThrowawayAAApeD95
999998,abir102


In [43]:
data.describe()

,UserNames
count,1000000
unique,1000000
top,anonquestion5
freq,1


In [44]:
datareq

,UserNames,Exist
0,kangerujack,0
1,zelbog,0
2,kickit2,0
3,throwaway24790,1
4,takashi37,0
...,...,...
49995,Gamerulf,0
49996,ZeStrife,0
49997,deppaotoko,0
49998,bballrox4717,0


In [45]:
datareq.describe()

,Exist
count,50000.000000
mean,0.042560
std,0.201865
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


### Results for Bloom filter with First_Hash function

In [46]:
b1 = B_Filter_handler_for_Second_Hash(1e6,0.09)
add(data, b1)
Accuracy_calculator(datareq, b1)
Is_available(datareq, b1)

________________________________________
B_Filter_handler_for_Second_Hash Report:
________________________________________
Size:  5011823
-----------------------------
bit array length 5011823
________________________________________


________________________________________
Adding usernames to bloom filter ... 
Adding process successfully completed
________________________________________


________________________________________
Accuracy_calculator Report:
________________________________________
Count 45691
-----------------------------
Accuracy rate 0.9146054222654953
-----------------------------
False positive rate:  0.09008822604220061
________________________________________
________________________________________
________________________________________


C:\Users\Mohammadmahdi\AppData\Local\Temp\ipykernel_10176\570280959.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Availability flag'][row]=1
C:\Users\Mohammadmahdi\AppData\Local\Temp\ipykernel_10176\570280959.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Availability flag'][row]=0
C:\Users\Mohammadmahdi\AppData\Local\Temp\ipykernel_10176\570280959.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Availab

In [47]:
datareq

,UserNames,Exist,Availability flag
0,kangerujack,0,1
1,zelbog,0,1
2,kickit2,0,1
3,throwaway24790,1,0
4,takashi37,0,1
...,...,...,...
49995,Gamerulf,0,1
49996,ZeStrife,0,1
49997,deppaotoko,0,1
49998,bballrox4717,0,1


In [48]:
class B_Filter_handler_for_First_Hash(object):
    def __init__(self, items_count, fp_prob):
        self.fp_prob = fp_prob
        self.size = self.get_size(items_count, fp_prob)
        self.hash_count = self.get_hash_count(self.size, items_count)
        print("________________________________________")
        print("B_Filter_handler_for_First_Hash Report:")
        print("________________________________________")
        print("Size: ",self.size)
        print("-----------------------------")
#         print("Hash function count: ",self.hash_count)
#         print("-----------------------------")
        self.bit_array = bitarray(self.size)
        print("bit array length",len(self.bit_array))
        print("________________________________________")
        self.bit_array.setall(0)
        self.set1 = set()

    def add(self, item):
        digests = []
        for i in [17,31,47,61]:
            digest = Second_Hash(item, i) % self.size
            digests.append(digest)
            self.set1.update({digest})
            self.bit_array[digest] = True

    def check(self, item):
        cv=0
        for j in [17,31,47,61]:
            digest = First_Hash(item, j) % self.size
            if self.bit_array[digest] == True:
                cv+=1
        if cv == 4:
            return True
        else :
             return False

    @classmethod
    def get_size(self, n, p):
        m = -(n * math.log(p))/(math.log(2)**2)
        return int(m)

    @classmethod
    def get_hash_count(self, m, n):
        k = (m/n) * math.log(2)
        return int(k)


In [49]:
b2 = B_Filter_handler_for_First_Hash(1e6,0.09)
add(data, b2)
Accuracy_calculator(datareq, b2)
Is_available(datareq, b2)

________________________________________
B_Filter_handler_for_First_Hash Report:
________________________________________
Size:  5011823
-----------------------------
bit array length 5011823
________________________________________


________________________________________
Adding usernames to bloom filter ... 
Adding process successfully completed
________________________________________


________________________________________
Accuracy_calculator Report:
________________________________________
Count 43622
-----------------------------
Accuracy rate 0.8782793730882139
-----------------------------
False positive rate:  0.09362044431912406
________________________________________
________________________________________
________________________________________


C:\Users\Mohammadmahdi\AppData\Local\Temp\ipykernel_10176\570280959.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Availability flag'][row]=1
C:\Users\Mohammadmahdi\AppData\Local\Temp\ipykernel_10176\570280959.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Availability flag'][row]=0
C:\Users\Mohammadmahdi\AppData\Local\Temp\ipykernel_10176\570280959.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Availab

In [50]:
datareq

,UserNames,Exist,Availability flag
0,kangerujack,0,1
1,zelbog,0,1
2,kickit2,0,1
3,throwaway24790,1,1
4,takashi37,0,1
...,...,...,...
49995,Gamerulf,0,1
49996,ZeStrife,0,1
49997,deppaotoko,0,0
49998,bballrox4717,0,1
